# COVID-19 Case Counts from Corona Data Scraper

**[Work in progress]**

This notebook creates a .csv file with cases for ingestion into the Knowledge Graph.

Data source: [Corona Data Scraper](https://coronadatascraper.com/)

Author: Peter Rose (pwrose@ucsd.edu)

In [1]:
import os
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
from pathlib import Path
import pandas as pd
import dateutil

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-b9d10363-6d59-4deb-9595-2cb904a99d1d/installation-4.1.0/import


In [4]:
df = pd.read_csv("https://coronadatascraper.com/timeseries.csv.zip",  dtype='str')

In [5]:
df['origLocation'] = df[['country', 'state', 'county', 'city']].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)

In [6]:
def get_aggregation_level(level):
    if level == 'country':
        return 'Country'
    elif level == 'state':
        return 'Admin1'
    elif level == 'county':
        return 'Admin2'
    elif level == 'city':
        return 'City'

In [7]:
df['aggregationLevel'] = df['level'].apply(lambda x: get_aggregation_level(x))

In [8]:
df = df.fillna('')

In [9]:
df = df[['date', 'origLocation', 'aggregationLevel', 'cases', 'deaths', 
         'recovered', 'tested', 'hospitalized', 'icu', 'population']]

In [12]:
df.head()

,date,origLocation,aggregationLevel,cases,deaths,recovered,tested,hospitalized,icu,population
0,2020-01-22,Andorra,Country,0,0,0,,,,76177
1,2020-01-23,Andorra,Country,0,0,0,,,,76177
2,2020-01-24,Andorra,Country,0,0,0,,,,76177
3,2020-01-25,Andorra,Country,0,0,0,,,,76177
4,2020-01-26,Andorra,Country,0,0,0,,,,76177


In [11]:
df.to_csv(NEO4J_IMPORT / "02b-CDSCases.csv", index=False)